In [ ]:
import cv2
import os
import numpy as np
from deepface import DeepFace

# Path to your folder with frames
frames_folder = "./happy"

# Store results in a dictionary
emotions_count = {}

# Loop through each frame in the folder
for frame_file in sorted(os.listdir(frames_folder)):
    if frame_file.endswith(".png"):  # Adjust if your frames are in JPEG or another format
        # Load the frame
        frame_path = os.path.join(frames_folder, frame_file)
        frame = cv2.imread(frame_path)
        
        # Check if the frame was loaded successfully
        if frame is None:
            print(f"Failed to load frame: {frame_file}")
            continue
        
        # Apply contrast enhancement and sharpening
        alpha = 1.5  # Contrast control
        beta = 20    # Brightness control
        enhanced_frame = cv2.convertScaleAbs(frame, alpha=alpha, beta=beta)
        
        # Sharpening filter
        kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        sharpened_frame = cv2.filter2D(enhanced_frame, -1, kernel)
        
        # Resize the frame for DeepFace
        resized_frame = cv2.resize(sharpened_frame, (224, 224))
        
        try:
            # Detect emotions with DeepFace
            analysis = DeepFace.analyze(resized_frame, actions=['emotion'], enforce_detection=False)
            
            # Check if analysis is a list and get the first element if it is
            if isinstance(analysis, list):
                analysis = analysis[0]
            
            # Extract dominant emotion
            emotion = analysis.get('dominant_emotion')
            if emotion:
                print(f"Detected {emotion} in frame: {frame_file}")
                
                # Update emotion count
                if emotion in emotions_count:
                    emotions_count[emotion] += 1
                else:
                    emotions_count[emotion] = 1
            else:
                print(f"No emotion detected in frame: {frame_file}")
                
        except Exception as e:
            print(f"Error processing frame {frame_file}: {e}")

# Determine the most common emotion, if any emotions were detected
if emotions_count:
    main_emotion = max(emotions_count, key=emotions_count.get)
    print(f"The main emotion detected in the video is: {main_emotion}")
else:
    print("No emotions were detected across all frames.")